**Author**: JW Debelius (justine.debelius@ki.se)<br>
**Date**: September 2021<br>
**Enviroment**: `qiime2-2021.8-dev`<br>
**Python version**: 3.8<br>
**Extra packages**: None<br>
**QIIME version**: 2021.8-dev<br>
**Extra Plugins**: q2-sidle (memory refactor); RESCRIPt (v. )

# Mock Community Data Preperation

## Background

This notebook will process four mock community samples sequenced with the Ion torrent metagenomic kit, originally published by [Barb et al, 2015](https://pubmed.ncbi.nlm.nih.gov/26829716/). In the original paper, the authors profiled four mock communites using the 6 primer proprietary Ion Torrent kit. This kit using 6 primer pairs to target 7 regions along the 16Ss gene with both forward and reverse reads possible.
* V2
* V3
* V4
* V67
* V8
* V9

In the original paper, the authors compared the performance for each region compared to the baseline. They clusstered sequences into de novo OTUs clustered with UPARSE; taxonomic assignment was made in QIIME 1 and then compared using abundance profiling, and deviation from published Shannon diversity. 

## Data download and avaliability

Sequences were deposided in SRA under accession SUB1054354. We used the [SRA CLI tools]() to download the sequences and the provided sample sheet. Sequence fastq files and the description were saved in the `mock` folder in the `data/input` directory. 

When they were deposited, the sequences were demultiplexed by sample, but not by region. The Ion Torrent kit produced reads for 12 regions (6forward and 6 reverse), meaning that to be able to use the sequences here, we need to split them into regions.

## Preprocessing Approach

For the sake of not hating everyone and everything in existing, we will take a somewhat less optimal approach and try to do a regional demux on  already denoised sequences, because otherwise both I and my computer willcry.         
So, Sequenced are prepared through the following steps: 

1. Sequences from each sample are filtered based on read length into batches based on read length. For this, we'll use 150, 200, 250, and 300 nt reads.
2. Import reads for each length using the manifest format 
3. Denoise the per-region sequences to their fixed read length using dada2-pyro which better handles the error profile. Trim the reads to the approriate read length during sequencing. 
4. Use RESCRIPt to orient the sequences so they have a consistent orientation
5. Align the reads against the reference sequences to seperate them into regions
6. Identify he regions through some kind of witch craft
7. Filter the sequences into samples and regions
8. Account for all the sequences that were lost during filtering, aligning, 
<!-- 9. Consider life choices and decide why there is such an obsession with mock communitites -->

## Set up

In [1]:
import os

import biom
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skbio

%matplotlib inline

from qiime2 import Artifact, Metadata, Visualization

In [2]:
input_dir = 'data/inputs/mock/' # modify if you've placed the files in another location
output_dir = 'data/output/mock/' # Change this if you want a different output
ref_dir = 'data/reference/' # Location of reference files

In [3]:
steps = {
    'import_seqs': {
        'run': True,
        'overwrite': True,
        'input_dir': input_dir,
        'output_dir': os.path.join(output_dir, '1.import'),
    },
    'denoise_seqs': {
        'run': True,
        'overwrite': True,
        'read_length': 200,
        'input_dir':  os.path.join(output_dir, '1.import'),
        'output_dir':  os.path.join(output_dir, '2.denoise'),
    },
    'cluster_orientation': {
        'run': True,
        'overwrite': True,
        'input_dir': os.path.join(output_dir, '2.denoise'),
        'output_dir': os.path.join(output_dir, '3.split_orientation'),
        'references': {'fwd': 'data/reference/gg_13_8_88/88_otus.qza',
                       'rev': 'data/reference/gg_13_8_88/88_otus_rc.qza'},
    },
    'generate_alignment_reference': {
        'run': True,
        'input_refs': {
            'fwd': 'data/reference/gg_13_8_88/gg_88_otus_aligned.qza',
            'rev': 'data/reference/gg_13_8_88/gg_88_otus_aligned_rc.qza',
            },
        'taxonomy_fp': 'data/reference/gg_13_8_88/88_otu_taxonomy.qza',
        'output_refs': {
            'fwd': 'data/reference/gg_13_8_88/gg_13_8_aligned_enterobacteraceae_fwd.qza',
            'rev': 'data/reference/gg_13_8_88/gg_13_8_aligned_enterobacteraceae_rev.qza',
        },
        'keep_group': 'f__Enterobacteriaceae',
    },
    'align_to_reference': {
        'run': True,
        'overwrite': True,
        'seq_input_dir':  os.path.join(output_dir, '3.split_orientation'),
        'table_input_dir': os.path.join(output_dir, '2.denoise'),
        'output_dir': os.path.join(output_dir, '4.aligned_to_ref'),
        'reference_groups': 'f__Enterobacteriaceae',
        'references': {'fwd':'data/reference/gg_13_8_88/gg_13_8_aligned_enterobacteraceae_fwd.qza',
                       'rev': 'data/reference/gg_13_8_88/gg_13_8_aligned_enterobacteraceae_fwd.qza',
                       },
    },
    'split_to_region': {
        'run': True,
        'overwrite': True,
        'input_map_dir': os.path.join(output_dir, '4.aligned_to_ref'),
        'input_data_dir': os.path.join(output_dir, '2.denoise'),
        'output_dir': os.path.join(output_dir, '5.regional_demux'),
    },
    'prepare_database': {
        'run': True,
        'overwrite': True,
        'input_seqs': os.path.join(ref_dir, 'silva/silva-128-99-seqs.qza'),
        'input_taxa': os.path.join(ref_dir, 'silva/silva-128-99-taxonomy.qza'),
        'input_alignment': os.path.join(ref_dir, 'silva/silva-128-99-aligned-seqs.qza'),
        'output_dir': os.path.join(output_dir, '6.database_alignment/'),
        'tmp_dir':   os.path.join(output_dir, 'data/output/6.database_alignment/tmp'),
    },
    'subsample_db': {
        'run': True,
        'overwrite': True,
        'fraction_subsample': 0.1,
        'seed': 1776,
        'input_alignment': os.path.join(output_dir, '6.database_alignment/silva-128-99-aligned-5-degen-bact-archea-only.qza'),
        'output_dir': os.path.join(output_dir, '6.database_alignment/'),
    },
    'align_regions_for_sub_db': {
        'run': True,
        'overwrite': True,
        'input_db_dir': os.path.join(output_dir, '6.database_alignment/'),
        'input_asv_dir': os.path.join(output_dir, '5.regional_demux'),
        'output_dir': os.path.join(output_dir, '7.regional_alignment'),
        'abundance_thresh': 1000,
    },
}

In [4]:
samples = [fp.split('.')[0] for fp in os.listdir(input_dir)
           if ('_1' in fp) & ('182' in fp)]

In [5]:
samples

['SRR2182222_1', 'SRR2182220_1', 'SRR2182221_1']

## Preprocessing

### Get preprocessing references

...|

### Import the data into QIIME 2 using a manifest format

Having split the data from each sample by read length, we'll 

In [10]:
def build_manifest(samples):
    """
    Builds a sample manifest for a specified read length
    """
    manifest = pd.DataFrame.from_dict(orient='index', data={
        sample: {
            'absolute-filepath': os.path.abspath(
                f'{manifest_input_dir}/{sample}.fastq.gz')
        }
        for sample in samples
    })
    manifest.index.set_names('sample-id', inplace=True)
    return Metadata(manifest)

In [11]:
if steps['import_seqs']['run']:
    manifest_input_dir = steps['import_seqs']['input_dir']
    manifest_output_dir = steps['import_seqs']['output_dir']
    manifest_overwrite = steps['import_seqs']['overwrite']

    os.makedirs(manifest_output_dir, exist_ok=manifest_overwrite)
    manifest_fp = f'{manifest_output_dir}/manifest.tsv'
    seqs_art_fp = f'{manifest_output_dir}/demux_reads.qza'
    seqs_vis_fp = f'{manifest_output_dir}/demux_reads.qzv'

    manifest = build_manifest(samples)
    manifest.save(manifest_fp)

    !qiime tools import \
      --type 'SampleData[SequencesWithQuality]' \
      --input-path $manifest_fp \
      --output-path $seqs_art_fp \
      --input-format SingleEndFastqManifestPhred33V2

    !qiime demux summarize \
     --i-data $seqs_art_fp \
     --o-visualization $seqs_vis_fp

Imported data/output/mock/1.import/manifest.tsv as SingleEndFastqManifestPhred33V2 to data/output/mock/1.import/demux_reads.qza
Saved Visualization to: data/output/mock/1.import/demux_reads.qzv


### Denoise sequences

The recommendation for Ion Torrent sequencing is to denoise using dada2-denoise pyro **[citeation neededd]** so we'll follow that

In [12]:
if steps['denoise_seqs']['run']:
    denoised_input = steps['denoise_seqs']['input_dir']
    denoised_output = steps['denoise_seqs']['output_dir']
    denoised_overwrite = steps['denoise_seqs']['overwrite']
    length = steps['denoise_seqs']['read_length']
    os.makedirs(denoised_output, exist_ok=denoised_overwrite)

    seqs_art_fp = f'{denoised_input}/demux_reads.qza'
    table_art_fp = f'{denoised_output}/table_.qza'
    rep_seq_art_fp = f'{denoised_output}/rep_seq.qza'
    stats_art_fp = f'{denoised_output}/denosing_stats.qza'
    table_viz_fp = f'{denoised_output}/table.qzv'
    stats_viz_fp = f'{denoised_output}/denosing_stats.qzv'

    !qiime dada2 denoise-pyro \
     --i-demultiplexed-seqs $seqs_art_fp \
     --p-trunc-len $length \
     --p-hashed-feature-ids \
     --p-n-threads 4 \
     --o-table $table_art_fp \
     --o-representative-sequences $rep_seq_art_fp \
     --o-denoising-stats $stats_art_fp

    !qiime metadata tabulate \
     --m-input-file $stats_art_fp \
     --o-visualization $stats_viz_fp

    !qiime feature-table summarize \
     --i-table $table_art_fp \
     --o-visualization $table_viz_fp

Saved FeatureTable[Frequency] to: data/output/mock/2.denoise/table_.qza
Saved FeatureData[Sequence] to: data/output/mock/2.denoise/rep_seq.qza
Saved SampleData[DADA2Stats] to: data/output/mock/2.denoise/denosing_stats.qza
Saved Visualization to: data/output/mock/2.denoise/denosing_stats.qzv
Saved Visualization to: data/output/mock/2.denoise/table.qzv


And now we have a set of full denoise tables with a fixed read lenght and mixed orientation and region reads. These are now ready for demultiplexing.

## Regional Demultiplexing


### Split the data into forward and reverse reads

We'll split the data by clustering the sequences and then filter to retain the clustered sequences

In [34]:
os.listdir(steps['cluster_orientation']['input_dir'])

['table.qzv',
 'denosing_stats.qzv',
 'table_.qza',
 'rep_seq.qza',
 'denosing_stats.qza']

In [24]:
if steps['cluster_orientation']['run']:
    cluster_input = steps['cluster_orientation']['input_dir']
    cluster_output = steps['cluster_orientation']['output_dir']
    cluster_overwrite = steps['cluster_orientation']['overwrite']
    cluster_references = steps['cluster_orientation']['references']
    cluster_rename = {'fwd': 'rev', 'rev': 'fwd'}
    os.makedirs(cluster_output, exist_ok=cluster_overwrite)
    
    for dir_, ref_seq_fp in cluster_references.items():    
        seqs_in_fp = f'{cluster_input}/rep_seq.qza'
        table_in_fp = f'{cluster_input}/table_.qza'
        cluster_otu_dir = f'{cluster_output}/clustered-seqs-{dir_}/'        
        seq_discard_fp = f'{cluster_otu_dir}/unmatched_sequences.qza'
        keep_seqs_fp = f'{cluster_output}/rep-seqs-{dir_}.qza'
        keep_table_fp = f'{cluster_output}/table-{dir_}.qza'
        
        # Clusters the sequences closed reference at a low percent identity
        !qiime vsearch cluster-features-closed-reference \
         --i-sequences $seqs_in_fp \
         --i-table $table_in_fp \
         --i-reference-sequences $ref_seq_fp \
         --p-perc-identity 0.85 \
         --output-dir $cluster_otu_dir

        # Filters the table against the discarded sequences
        !qiime feature-table filter-seqs \
         --i-data $seqs_in_fp \
         --m-metadata-file $seq_discard_fp \
         --p-exclude-ids \
         --o-filtered-data $keep_seqs_fp
        
        !qiime feature-table filter-features \
         --i-table $table_in_fp \
         --m-metadata-file  $keep_seqs_fp \
         --o-filtered-table $keep_table_fp
        
        !rm -r $cluster_otu_dir

rm: data/output/mock/3.split_orientation/clustered-seqs-fwd/: No such file or directory
Saved FeatureTable[Frequency] to: data/output/mock/3.split_orientation/clustered-seqs-fwd/clustered_table.qza
Saved FeatureData[Sequence] to: data/output/mock/3.split_orientation/clustered-seqs-fwd/clustered_sequences.qza
Saved FeatureData[Sequence] to: data/output/mock/3.split_orientation/clustered-seqs-fwd/unmatched_sequences.qza
Saved FeatureData[Sequence] to: data/output/mock/3.split_orientation/rep-seqs-fwd.qza
Saved FeatureTable[Frequency] to: data/output/mock/3.split_orientation/table-fwd.qza
rm: data/output/mock/3.split_orientation/clustered-seqs-rev/: No such file or directory
Saved FeatureTable[Frequency] to: data/output/mock/3.split_orientation/clustered-seqs-rev/clustered_table.qza
Saved FeatureData[Sequence] to: data/output/mock/3.split_orientation/clustered-seqs-rev/clustered_sequences.qza
Saved FeatureData[Sequence] to: data/output/mock/3.split_orientation/clustered-seqs-rev/unmatched

### Generate sub alignment references

In [54]:
if steps['generate_alignment_reference']['run']: 
    input_refs = steps['generate_alignment_reference']['input_refs']
    output_refs = steps['generate_alignment_reference']['output_refs']
    taxonomy_fp = steps['generate_alignment_reference']['taxonomy_fp']
    keep_group = steps['generate_alignment_reference']['keep_group']

    for dir_, input_ in input_refs.items():
        output = output_refs[dir_]
        if not os.path.exists(output):
            !qiime taxa filter-seqs \
             --i-sequences $input_ \
             --i-taxonomy $taxonomy_fp \
             --p-include $keep_group \
             --o-filtered-sequences $output

### Align the oriented reference data and finds the starting positions

In [55]:
!qiime dev refresh-cache

QIIME is caching your current deployment for improved performance. This may take a few moments and should only happen once per deployment.


In [62]:
if steps['align_to_reference']['run']: 
    align_repseq_dir = steps['align_to_reference']['seq_input_dir']
    align_table_dir = steps['align_to_reference']['table_input_dir']
    align_output_dir = steps['align_to_reference']['output_dir']
    align_overwrite = steps['align_to_reference']['overwrite']
    align_references =  steps['align_to_reference']['references']

    os.makedirs(align_output_dir, exist_ok=align_overwrite)

    for dir_, ref_alignment_fp in align_references.items():
        rep_seq_fp = f'{align_repseq_dir}/rep-seqs-{dir_}.qza'
        table_fp = f'{align_table_dir}/table_.qza'

        alignment_fp = \
            f'{align_output_dir}/rep_set_aligned_{dir_}.qza'
        output_pos_art = \
            f'{align_output_dir}/starts-{dir_}.qza'
        output_pos_viz = \
            f'{align_output_dir}/starts-{dir_}.qzv'

        !qiime sidle map-alignment-positions \
         --i-alignment $ref_alignment_fp \
         --i-sequences $rep_seq_fp \
         --i-table $table_fp \
         --p-direction $dir_ \
         --p-no-add-fragments \
         --p-colormap viridis \
         --o-expanded-alignment $alignment_fp \
         --o-position-summary $output_pos_art \
         --o-position-map $output_pos_viz

Saved FeatureData[AlignedSequence] to: data/output/mock/4.aligned_to_ref/rep_set_aligned_fwd.qza
Saved FeatureData[AlignmentPosSummary] to: data/output/mock/4.aligned_to_ref/starts-fwd.qza
Saved Visualization to: data/output/mock/4.aligned_to_ref/starts-fwd.qzv
Saved FeatureData[AlignedSequence] to: data/output/mock/4.aligned_to_ref/rep_set_aligned_rev.qza
Saved FeatureData[AlignmentPosSummary] to: data/output/mock/4.aligned_to_ref/starts-rev.qza
Saved Visualization to: data/output/mock/4.aligned_to_ref/starts-rev.qzv


In [205]:
# from qiime2 import Visualization
# Visualization.load('data/output/mock/4.aligned_to_ref/starts-fwd.qzv')

In [206]:
# Visualization.load('data/output/mock/4.aligned_to_ref/starts-rev.qzv')

### Extracts the starting position from the alignment

### Regional Demultiplexing 

Based on the visualizaation, I can infer a set of starting positions. For the forward reads, I looked at feature with at least 2 sequences and a maxium relative abundnce of at least 1000 sequences. This gives me starting at 69, 303, 508, 914, 1043, 1285 for the forward reads. The reverse positions are a little harder because of the weird split in thata block around 400. But, we also read the starting position backward, so maybe it's not so weird? The reverse reads end up at 349, 534, 805, 1134, 1302, and 1455.

In [7]:
Artifact.load('data/output/mock/4.aligned_to_ref/starts-fwd.qza').view(Metadata)

Metadata
--------
137 IDs x 3 columns
starting-position: ColumnProperties(type='categorical')
sequence-counts:   ColumnProperties(type='numeric')
direction:         ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation.

In [6]:
first_positions = {'fwd': [ 69, 308, 516, 926, 1054, 1301],
                   'rev': [340, 541, 815, 1153, 1313, 1466]
                   }

I'll use those positions to split the data.

In [27]:
if steps['split_to_region']['run']: 
    region_demux_overwrite = steps['split_to_region']['overwrite']
    region_demux_meta_dir = steps['split_to_region']['input_map_dir']
    region_demux_data_dir = steps['split_to_region']['input_data_dir']
    region_demux_output_dir = steps['split_to_region']['output_dir']

    os.makedirs(region_demux_output_dir, exist_ok=region_demux_overwrite)

    for dir_, positions in first_positions.items():
        for pos in positions:
            input_table_fp = f'{region_demux_data_dir}/table_.qza'
            input_rep_seq_fp = \
                f'{region_demux_data_dir}/rep_seq.qza'

            meta_fp = f'{region_demux_meta_dir}/starts-{dir_}.qza'
            
            table_fp = \
                f'{region_demux_output_dir}/table-{dir_}-{pos}.qza'
            rep_seq_fp = \
                f'{region_demux_output_dir}/rep-seq-{dir_}-{pos}.qza'
            table_summary_fp = \
                 f'{region_demux_output_dir}/table-{dir_}-{pos}.qzv'
            
            where = f'[starting-position]="{pos}"'

            !qiime feature-table filter-features \
             --i-table $input_table_fp \
             --m-metadata-file $meta_fp \
             --p-where $where \
             --o-filtered-table $table_fp

            !qiime feature-table filter-seqs \
             --i-data $input_rep_seq_fp \
             --i-table $table_fp \
             --o-filtered-data $rep_seq_fp

            !qiime feature-table summarize \
             --i-table $table_fp \
             --o-visualization $table_summary_fp

            if dir_ == 'rev':
                !qiime sidle reverse-complement-sequence \
                 --i-sequence $rep_seq_fp \
                 --o-reverse-complement $rep_seq_fp

data/output/mock/5.regional_demux/table-fwd-69.qza
Saved FeatureTable[Frequency] to: data/output/mock/5.regional_demux/table-fwd-69.qza
Saved FeatureData[Sequence] to: data/output/mock/5.regional_demux/rep-seq-fwd-69.qza
Saved Visualization to: data/output/mock/5.regional_demux/table-fwd-69.qzv
data/output/mock/5.regional_demux/table-fwd-308.qza
Saved FeatureTable[Frequency] to: data/output/mock/5.regional_demux/table-fwd-308.qza
Saved FeatureData[Sequence] to: data/output/mock/5.regional_demux/rep-seq-fwd-308.qza
Saved Visualization to: data/output/mock/5.regional_demux/table-fwd-308.qzv
data/output/mock/5.regional_demux/table-fwd-516.qza
Saved FeatureTable[Frequency] to: data/output/mock/5.regional_demux/table-fwd-516.qza
Saved FeatureData[Sequence] to: data/output/mock/5.regional_demux/rep-seq-fwd-516.qza
Saved Visualization to: data/output/mock/5.regional_demux/table-fwd-516.qzv
data/output/mock/5.regional_demux/table-fwd-926.qza
Saved FeatureTable[Frequency] to: data/output/mock/5

## Accounting

Finally, I'd like to determine how many sequences were lost and where they were lost. For this, I need the dada2 stats... 

In [29]:
dada2_summaries = \
    Artifact.load(f'data/output/mock/2.denoise/denosing_stats.qza')


# Reconstruction

## Database

We're going to start by pre-filtering the database. For reasons I'm not quite sure I understand, it's not possible to do filtering on the alignmnet sequences directly, so I'm going to filter the reference sequences the way I want them, use *those* to filter the alignmnet set, and then generate a subset of aligned sequences I can use for analysis.
<!-- 
To extract the positions, I'm going to use the most abundant features from each region and align them against the full reference set. For this analysis, I want to work with Silva 128 [cite]. I picked 128 specifically to be able to do phylogenetic tree reconstruction, although at this point, the phylogenny doesns't really matter, so I guess it's more for consistency with the simulated data ¯\\\_(ツ)\_/¯. I think we've demonstrated the use o fmultiple databases sucessfully, if not I can switch this to greengenes -->

In [12]:
if steps['prepare_database']['run']:
    prep_db_overwrite = steps['prepare_database']['overwrite']
    prep_db_input_seqs = steps['prepare_database']['input_seqs']
    prep_db_input_taxa = steps['prepare_database']['input_taxa']
    prep_db_input_alignment = steps['prepare_database']['input_alignment']
    prep_db_output = steps['prepare_database']['output_dir']
    prep_db_tmp_dir = steps['prepare_database']['tmp_dir']

    os.makedirs(prep_db_output, exist_ok=prep_db_overwrite)
    os.makedirs(prep_db_tmp_dir, exist_ok=True)

    !qiime rescript cull-seqs \
     --i-sequences $prep_db_input_seqs \
     --p-num-degenerates 6 \
     --o-clean-sequences $prep_db_tmp_dir/silva-128-99-low-degen.qza 

    !qiime taxa filter-seqs \
     --i-sequences $prep_db_tmp_dir/silva-128-99-low-degen.qza \
     --i-taxonomy $prep_db_input_taxa \
     --p-include 'D_0__Bact,D_0__Arch' \
     --o-filtered-sequences $prep_db_tmp_dir/silva-128-99-low-degen-bact-archea.qza

    !qiime feature-table filter-seqs \
     --i-data $prep_db_input_alignment \
     --m-metadata-file $prep_db_tmp_dir/silva-128-99-low-degen-bact-archea.qza \
     --o-filtered-data $prep_db_output/silva-128-99-aligned-5-degen-bact-archea-only.qza

In [33]:
total_alignment = Artifact.load(f'{prep_db_output}/silva-128-99-aligned-5-degen-bact-archea-only.qza')

In [46]:
import daskjoin

In [44]:
# total_align_mas.consensus()

And then, we'll subsample the reference to 10% to fit the data

In [13]:
if steps['subsample_db']['run']:
    subsample_db_overwrite = steps['subsample_db']['overwrite']
    subsample_db_input_alignment = steps['subsample_db']['input_alignment']
    subsample_db_fraction = steps['subsample_db']['fraction_subsample']
    subsample_db_seed = steps['subsample_db']['seed']
    subsample_db_output_dir =  steps['subsample_db']['output_dir']
    
    os.makedirs(subsample_db_output_dir, exist_ok=subsample_db_overwrite)
    
    output_alignment = (f'{subsample_db_output_dir}silva-128-99-aligned-'
                        f'5-degen-bact-archea-only-{subsample_db_fraction}.qza')
    
    !qiime rescript subsample-fasta \
     --i-sequences $subsample_db_input_alignment \
     --p-random-seed $subsample_db_seed \
     --p-subsample-size $subsample_db_fraction \
     --o-sample-sequences $output_alignment

Saved FeatureData[AlignedSequence] to: data/output/mock/6.database_alignment/silva-128-99-aligned-5-degen-bact-archea-only-0.1.qza


# Performs regional alignment

In [ ]:
steps['align_regions_for_sub_db']

{'run': True,
 'overwrite': True,
 'input_db_dir': 'data/output/mock/6.database_alignment/',
 'input_asv_dir': 'data/output/mock/5.regional_demux',
 'output_dir': 'data/output/mock/7.regional_alignment',
 'abundance_thresh': 1000}

In [18]:
align_regions_db_input = steps['align_regions_for_sub_db']['input_db_dir']
align_asv_input = steps['align_regions_for_sub_db']['input_asv_dir']

align_regions_overwrite = steps['align_regions_for_sub_db']['overwrite']
align_asv_out = steps['align_regions_for_sub_db']['output_dir']
os.makedirs(align_asv_out, exist_ok=align_regions_overwrite)

ref_align = f'{align_regions_db_input}/silva-128-99-aligned-5-degen-bact-archea-only-0.1.qza'

dir_ = 'fwd'
pos = 308
thresh = steps['align_regions_for_sub_db']['abundance_thresh']

In [20]:
region_dir = f'{align_asv_out}/{dir_}-{pos}'
os.makedirs(region_dir)

full_table_fp = f'{align_asv_input}/table-{dir_}-{pos}.qza'
full_repseq_fp = f'{align_asv_input}/rep-seq-{dir_}-{pos}.qza'

!qiime feature-table filter-features \
 --i-table $full_table_fp \
 --p-min-frequency $thresh \
 --o-filtered-table $region_dir/table.qza

!qiime feature-table filter-seqs \
 --i-data $full_repseq_fp \
 --i-table $region_dir/table.qza \
 --o-filtered-data 

Saved FeatureTable[Frequency] to: data/output/mock/7.regional_alignment/fwd-308/table.qza
Saved FeatureData[Sequence] to: data/output/mock/7.regional_alignment/fwd-308/rep-seq.qza


And then, I want to preform local regional alignment against the reference for each region.

In [25]:
!qiime alignment mafft-add \
 --i-alignment $ref_align \
 --i-sequences $region_dir/rep-seq.qza \
 --o-expanded-alignment $region_dir/expanded-alignment.qza \
 --verbose

Running external command line application. This may print messages to stdout and/or stderr.
The command being run is below. This command cannot be manually re-run as it will depend on temporary files that no longer exist.

Command: mafft --preservecase --inputorder --thread 1 --add /var/folders/bw/q064ds0d2795_6mxnrssf0l1gkw0rj/T/qiime2-archive-bontauj9/82e567c5-e1bc-47a8-93bb-7fc29191ba1e/data/dna-sequences.fasta /var/folders/bw/q064ds0d2795_6mxnrssf0l1gkw0rj/T/qiime2-archive-5zi4c4ja/bd28e29f-6d62-4d6f-b643-b4dc0d932233/data/aligned-dna-sequences.fasta

inputfile = orig
32423 x 34482 - 200 d
nadd = 15
nthread = 1
nthreadpair = 1
nthreadtb = 1
ppenalty_ex = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..

There are 46289768 ambiguous characters.
 32401 / 32423 (thread    0)
done.

Constructing a UPGMA tree (efffree=0) ... 
 32420 / 32423
done.

Progressive alignment 1/2... 
STEP   313 

Aligns the filtered sequences against the reference

In [12]:
from qiime2 import Artifact
import pandas as pd
import skbio
import seaborn as sn
import numpy as np

%matplotlib inline

alignment = Artifact.load('data/output/mock/7.regional_alignment/fwd-308/expanded-alignment.qza')
rep_seqs = Artifact.load('data/output/mock/7.regional_alignment/fwd-308/rep-seq.qza')

In [9]:
msa = alignment.view(skbio.TabularMSA)

In [10]:
alignment.view(pd.Series).index

Index(['GDUJ01044859.40.1504', 'GBGZ01012382.1360.2968', 'GCET01095040.1.1412',
       'GDZX01002907.1.1364', 'GDZX01002911.1.1386', 'GDZX01002922.1.1373',
       'GAMG01019100.45.1508', 'GBXT01022816.16.1503', 'GABH01006691.590.2223',
       'GCZP01012215.295.1631',
       ...
       '99706e7fabeb8d8c7dc683217ce41bc4', '8f67a68b8f2de0845909eafee91923a7',
       'ff8d2d1a7eea0ae130f6ad7a528c4156', '4c82323160354936a0074da200539ba4',
       '221282155f99e7c5c1db27bb52739487', '720206598fb2c03161ed5bbbc0a92591',
       'c961c4d842423c92318a287ce0c731d4', '3f0aaf2e8a5d2ee6f2500d8368b82174',
       '8a50ccc5ea45f5ae5519694941ef1e62', '41549554bb33cc37ae5b2c2857e3c66d'],
      dtype='object', length=32423)

In [14]:
sub_align = alignment.view(pd.Series).loc[rep_seqs.view(pd.Series).index]

In [17]:
sub_df = sub_align.astype(str).apply(lambda x: pd.Series(list(x)))

In [19]:
aligned_pos = sub_df.replace({'.': np.nan, '-': np.nan}).dropna(how='all', axis=1)
aligned_pos

,4730,4732,4734,4735,4738,4750,4751,4755,4757,4758,...,7676,8817,8819,8821,8822,8825,8827,8831,8834,8836
da0ed08fa0c973902eb97f8ae504878d,A,C,T,G,A,G,A,C,A,C,...,T,G,C,C,NaN,NaN,NaN,NaN,NaN,NaN
43530a17ab9e2e1253699f23f0492152,A,C,T,G,A,G,A,C,A,C,...,T,G,C,C,A,NaN,NaN,NaN,NaN,NaN
d4f3b1f7e9e1a4d6ccf9584a14fb6005,A,C,T,G,A,G,A,C,A,C,...,T,G,C,C,A,G,NaN,NaN,NaN,NaN
2dd11b0e435f890bae0d76c21af09312,A,C,T,G,A,G,A,C,A,C,...,T,G,C,C,NaN,NaN,NaN,NaN,NaN,NaN
5f870adb1d762390b1b58632c2e0bf89,A,C,T,G,A,G,A,C,A,C,...,T,G,C,C,NaN,NaN,NaN,NaN,NaN,NaN
99706e7fabeb8d8c7dc683217ce41bc4,A,C,T,G,A,G,A,C,A,C,...,T,G,C,C,A,NaN,NaN,NaN,NaN,NaN
8f67a68b8f2de0845909eafee91923a7,A,C,T,G,A,G,A,C,A,C,...,T,G,C,C,NaN,NaN,NaN,NaN,NaN,NaN
ff8d2d1a7eea0ae130f6ad7a528c4156,A,C,T,G,A,G,A,C,A,C,...,T,G,C,C,A,NaN,NaN,NaN,NaN,NaN
4c82323160354936a0074da200539ba4,A,C,T,G,A,G,A,C,A,C,...,T,G,C,C,A,NaN,NaN,NaN,NaN,NaN
221282155f99e7c5c1db27bb52739487,A,C,T,G,A,G,A,C,A,C,...,T,G,C,C,A,NaN,NaN,NaN,NaN,NaN


In [181]:
# aligned_pos =a 

In [185]:
print(aligned_pos.columns.min(), aligned_pos.columns.max())

4730 8836


In [190]:
!mkdir -p 'data/output/mock/7.regional-reference/'

In [20]:
!qiime rescript trim-alignment --help
#  --i-aligned-sequences data/reference/silva/silva-128-99-aligned-seqs.qza \
#  --p-position-start 4730 \
#  --p-position-end 8836 \
#  --o-trimmed-sequences data/output/mock/7.regional-reference/silva-128-99-aligned-308-fwd.qza

Usage: qiime rescript trim-alignment [OPTIONS]

  Trim an existing alignment based on provided primers or specific, pre-
  defined positions. Primers take precedence over the positions,i.e. if both
  are provided, positions will be ignored.When using primers in combination
  with a DNA alignment, a new alignment will be generated to locate primer
  positions. Subsequently, start (5'-most) and end (3'-most) position from
  fwd and rev primer located within the new alignment is identified and used
  for slicing the original alignment.

Inputs:
  --i-aligned-sequences ARTIFACT FeatureData[AlignedSequence]
                       Aligned DNA sequences.                       [required]
Parameters:
  --p-primer-fwd TEXT  Forward primer used to find the start position for
                       alignment trimming.                          [optional]
  --p-primer-rev TEXT  Reverse primer used to find the end position for
                       alignment trimming.                          [optio

In [194]:
!qiime rescript degap-seqs \
 --i-aligned-sequences data/output/mock/7.regional-reference/silva-128-99-aligned-308-fwd.qza \
 --o-degapped-sequences data/output/mock/7.regional-reference/silva-128-99-308-fwd.qza

Saved FeatureData[Sequence] to: data/output/mock/7.regional-reference/silva-128-99-308-fwd.qza


In [198]:
!qiime feature-table tabulate-seqs \
 --i-data data/output/mock/7.regional-reference/silva-128-99-308-fwd.qza \
 --o-visualization data/output/mock/7.regional-reference/silva-128-99-308-fwd.qzv

Saved Visualization to: data/output/mock/7.regional-reference/silva-128-99-308-fwd.qzv


In [21]:
!qiime rescript degap-seqs --help

Usage: qiime rescript degap-seqs [OPTIONS]

  This method converts aligned DNA sequences to unaligned DNA sequences by
  removing gaps ("-") and missing data (".") characters from the sequences.
  Essentially, 'unaligning' the sequences.

Inputs:
  --i-aligned-sequences ARTIFACT FeatureData[AlignedSequence]
                          Aligned DNA Sequences to be degapped.     [required]
Parameters:
  --p-min-length INTEGER  Minimum length of sequence to be returned after
    Range(1, None)        degapping.                              [default: 1]
Outputs:
  --o-degapped-sequences ARTIFACT FeatureData[Sequence]
                          The resulting unaligned (degapped) DNA sequences.
                                                                    [required]
Miscellaneous:
  --output-dir PATH       Output unspecified results to a directory
  --verbose / --quiet     Display verbose output to stdout and/or stderr
                          during execution of this action. Or silence o

In [24]:
!qiime rescript filter-seqs-length --help

Usage: qiime rescript filter-seqs-length [OPTIONS]

  Filter sequences by length with VSEARCH. For a combination of global and
  conditional taxonomic filtering, see filter_seqs_length_by_taxon.

Inputs:
  --i-sequences ARTIFACT FeatureData[Sequence]
                          Sequences to be filtered by length.       [required]
Parameters:
  --p-global-min INTEGER  The minimum length threshold for filtering all
    Range(1, None)        sequences. Any sequence shorter than this length
                          will be removed.                          [optional]
  --p-global-max INTEGER  The maximum length threshold for filtering all
    Range(1, None)        sequences. Any sequence longer than this length will
                          be removed.                               [optional]
  --p-threads INTEGER     Number of computation threads to use (1 to 256).
    Range(1, 256)         The number of threads should be lesser or equal to
                          the number of availabl